# Broadcasting, Pivoting and Melting in Pandas

In [30]:
import pandas as pd
import numpy as np

ttb_oct_2021 = pd.read_excel("./data/Statistical_Report_Beer_October_2021.xlsx")

ttb_oct_2021


,DEPARTMENT OF THE TREASURY,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7
0,ALCOHOL AND TOBACCO TAX AND TRADE BUREAU,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,STATISTICAL REPORT - BEER,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Reporting Period: October 2021,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Report Date: 12JAN2022,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Report Symbol: TTB S 5130-10-2021,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,1 Barrel is equivalent to 31 gallons.,NaN,NaN,NaN,Current\nMonth,Prior Year\nCurrent\nMonth,Current\nYear\nCumulative\nto Date,Prior Year\nCumulative\nto Date
8,Manufacture\nOf Beer ...,Production,Production,Production,14232172,14249879,152978238,151960985
9,NaN,Removals,Taxable*,In bottles & cans,12708198,12901313,130557245,133253958


In [31]:
# Extract the relevant rows and columns
dates = ttb_oct_2021.iloc[[3, 4], 0]
tmp = ttb_oct_2021.iloc[[7, 8, 18], 3:].reset_index(drop=True)

# Clean the column names by removing "Unnamed: n" and newlines
tmp.columns = tmp.iloc[0].fillna("")
tmp = tmp[1:]
# Clean the column names
tmp.columns = tmp.columns.str.replace(r'Unnamed:\s*\d+', '', regex=True).str.strip()
tmp.columns = tmp.columns.str.replace(r'\n', ' ', regex=True)

# Rename the empty column
tmp = tmp.rename(columns={"": "Metrics"})

tmp = tmp.T.reset_index(drop=False)
tmp.columns = tmp.iloc[0]
tmp = tmp[1:]
tmp['Total Sales'] = tmp['Production'] - tmp['Stocks On Hand End-of-Month']
tmp.iloc[2:,3] = np.nan
tmp


,Metrics,Production,Stocks On Hand End-of-Month,Total Sales
1,Current Month,14232172,10798720,3433452
2,Prior Year Current Month,14249879,9555100,4694779
3,Current Year Cumulative to Date,152978238,NaN,NaN
4,Prior Year Cumulative to Date,151960985,NaN,NaN


In [33]:
dates = ttb_oct_2021.iloc[[3, 4], 0].str.split(":").apply(lambda x: [s.strip() for s in x])
dates_dict = dict(tuple(dates))

dates_dict = {k:pd.to_datetime(v) for k,v in dates_dict.items()}
for k,v in dates_dict.items():
    tmp[k] = v

tmp.to_markdown("./ttb.md", float_format=".0f")